In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

In [ ]:
# Define transforms
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Define the dataset and dataloader
train_dataset = datasets.ImageFolder('/Users/srinijacherivirala/Desktop/train/', transform=transform_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.ImageFolder('/Users/srinijacherivirala/Desktop/test/', transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# Define the model
model = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Conv2d(64, 192, kernel_size=5, padding=2),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Conv2d(192, 384, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(384, 256, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.Conv2d(256, 256, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Flatten(),
    nn.Linear(256 * 6 * 6, 4096),
    nn.ReLU(inplace=True),
    nn.Linear(4096, 4096),
    nn.ReLU(inplace=True),
    nn.Linear(4096, 2)
)

In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [10]:
# Train the model
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # Get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

[1,   100] loss: 0.477
[2,   100] loss: 0.371
[3,   100] loss: 0.344
[4,   100] loss: 0.341
[5,   100] loss: 0.333
[6,   100] loss: 0.336
[7,   100] loss: 0.335
[8,   100] loss: 0.331


KeyboardInterrupt: 

USING RESNET34 BELOW

In [18]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms, models

# Load a pre-trained ResNet-34 model and replace the final fully-connected layer
model = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch: %d, Loss: %.3f' % (epoch + 1, running_loss / len(train_loader)))

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /Users/srinijacherivirala/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)>

GENERAL TEST CELL

In [14]:
device = torch.device('cpu')

# Put the model in evaluation mode
model.eval()

# Initialize variables to keep track of loss and accuracy
test_loss = 0
accuracy = 0

# Loop over the test data in batches
for images, labels in test_loader:
    # Move the data to the device
    images, labels = images.to(device), labels.to(device)

    # Forward pass
    output = model(images)

    # Calculate the loss
    loss = criterion(output, labels)

    # Update the test loss
    test_loss += loss.item()*images.size(0)

    # Calculate the accuracy
    _, pred = torch.max(output, 1)
    correct_tensor = pred.eq(labels.data.view_as(pred))
    accuracy += torch.mean(correct_tensor.type(torch.FloatTensor)).item()*images.size(0)

# Calculate the average loss and accuracy
test_loss = test_loss/len(test_loader.dataset)
accuracy = accuracy/len(test_loader.dataset)

print('Test Loss: {:.6f}\n'.format(test_loss))
print('Test Accuracy: {:.2f}%\n'.format(accuracy*100))

Test Loss: 0.328470

Test Accuracy: 87.98%

